In [298]:
#######################
# Project 3
# ESE 572
# Layer 2 Channel Coding
#######################

# Choose a probability of bit error to use when simulating the receiving end.

# prob = 0.1
prob = 0.01

In [299]:
#######################
# Step 1
#######################

# Function to convert string to binary using ASCII encoding
def string_to_binary(string):
    binary = ''
    binaryS = ''
    binaryC = ''
    counter = 0
    for char in string:
        ascii_code = ord(char) # Get ASCII code of character
        if counter == 24:
            binary += format(ascii_code, '08b')
            binaryS += format(ascii_code, '08b') + ' '
            binaryC += format(ascii_code, '08b') + '\n'
            counter = 0
        else:
            binary += format(ascii_code, '08b')
            binaryS += format(ascii_code, '08b') + ' '
            binaryC += format(ascii_code, '08b')
            counter += 1
    return binary, binaryS, binaryC

# Load the document
filename = 'input.txt'
text = ''
with open(filename,'r') as i:
    text += i.readline()
    
binary_data, binaryS_data, binaryC_data = string_to_binary(text)

# binary_data is single line
# binaryS_data contain spaces to delimitate each character
# binaryC_data is split as chunks of 200 bits


In [300]:
#######################
# Step 2
#######################

# Create CRC for g(D) = [D16 + D12 + D5 + 1]
crc_str = '10001000000100001' # divisor
int_crc_str = int(crc_str,2)
frames = []

binary_chunks = binaryC_data.split('\n')
for chunk in binary_chunks[:-1]:
    temp = chunk + "0000000000000000"
    temp = int(temp,2)
    crc = temp % int_crc_str
    crc = format(crc, '016b')
    if len(crc) > 16:
        crc = crc[-16:]
    frames.append(chunk + crc)

print(len(frames[0]))

216


In [301]:
#######################
# Step 3
#######################
## Channel Coding **NEW**

import numpy as np

def encode_message(msg, matrix):
    code = np.mod(np.dot(msg, matrix), 2)
    return code

def decode_message(code, valid_codewords):
    incorrect_vec = []
    incorrect = 0
    for codeword in valid_codewords:
        for ind in range(len(codeword)):
            if codeword[ind] != code[ind]:
                incorrect += 1
        incorrect_vec.append(incorrect)
        incorrect = 0
    closest = np.min(incorrect_vec)
    closest = np.where(incorrect_vec == closest)[0][0]
    decoded_msg = valid_data[closest]
    return decoded_msg

G = np.array([[1,1,1,1,1,1,1,1],
              [0,0,0,0,1,1,1,1],
              [0,0,1,1,0,0,1,1],
              [0,1,0,1,0,1,0,1]])

valid_data = [[0,0,0,0], [0,0,0,1], [0,0,1,0], [0,0,1,1],
              [0,1,0,0], [0,1,0,1], [0,1,1,0], [0,1,1,1],
              [1,0,0,0], [1,0,0,1], [1,0,1,0], [1,0,1,1],
              [1,1,0,0], [1,1,0,1], [1,1,1,0], [1,1,1,1]]

valid_codewords = []
for data in valid_data:
    valid_codewords.append(encode_message(data,G))
print(valid_codewords)

# Example
msg = np.array([0, 1, 1, 1])
print("Sent Message:", msg)
print()

code = encode_message(msg, G)
print("Encoded code:", code)
print()

decoded_msg = decode_message(code, valid_codewords)
print("Decoded message:", decoded_msg)
# End Example

def array_to_string(array):
    output = ""
    for each in array:
        output += str(each)
    return output

def string_to_array(string):
    output = []
    for each in string:
        output.append(int(each))
    output = np.array(output)
    return output

def RM_string(input_string):
    output_string = ""
    ind = 0
    while(ind <= len(input_string)):
        dat = input_string[ind:ind+4]
        data_string = string_to_array(dat)
        if data_string.size != 0:
            data_string = data_string.reshape((1,4))
            encoded = encode_message(data_string,G)[0]
            output_string += array_to_string(encoded)
        else:
            output_string += str(dat[-1:])
        ind += 4
    return output_string

binaryRM = []
for frame in frames:
    binaryRM.append(RM_string(frame))
    
print(len(binaryRM[0]))
print(binaryRM[0])

[array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), array([0, 1, 0, 1, 0, 1, 0, 1], dtype=int32), array([0, 0, 1, 1, 0, 0, 1, 1], dtype=int32), array([0, 1, 1, 0, 0, 1, 1, 0], dtype=int32), array([0, 0, 0, 0, 1, 1, 1, 1], dtype=int32), array([0, 1, 0, 1, 1, 0, 1, 0], dtype=int32), array([0, 0, 1, 1, 1, 1, 0, 0], dtype=int32), array([0, 1, 1, 0, 1, 0, 0, 1], dtype=int32), array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int32), array([1, 0, 1, 0, 1, 0, 1, 0], dtype=int32), array([1, 1, 0, 0, 1, 1, 0, 0], dtype=int32), array([1, 0, 0, 1, 1, 0, 0, 1], dtype=int32), array([1, 1, 1, 1, 0, 0, 0, 0], dtype=int32), array([1, 0, 1, 0, 0, 1, 0, 1], dtype=int32), array([1, 1, 0, 0, 0, 0, 1, 1], dtype=int32), array([1, 0, 0, 1, 0, 1, 1, 0], dtype=int32)]
Sent Message: [0 1 1 1]

Encoded code: [0 1 1 0 1 0 0 1]

Decoded message: [0, 1, 1, 1]
432
000011111010101000111100110000110011001100000000011001100101010101100110111111110110011001011010011001100110011000110011111100000011001100000000011010010000000001101001001

In [321]:
#######################
# Step 4
#######################
#binRM = binaryRM[0]
# flag = '01111110'

binRM= binaryRM[0]

flag = '000111111111111111111000' # tripled
framed_frames = []
bitstuff=0
#for binRM in binaryRM:
i = 0

while(i <= (len(binRM) - 18)):
    if binRM[i:i+17] == '11111111111111111':
        binRM = binRM[:i+16] + '0' + binRM[i+16:] # bit stuff after any string of five 1s
        bitstuff+=1
    i += 8
framed_frames.append(flag + binRM + flag)
print(i)
#frame0 = array_to_string(code)
#frame0 = flag+frame0+flag
#print(frame0)
print(framed_frames[0])
#framed_frames.append(flag) # after last FEC frame, insert another flag
print('bits stuffed: '+str(bitstuff))
print(framed_frames)

416
000111111111111111111000000011111010101000111100110000110011001100000000011001100101010101100110111111110110011001011010011001100110011000110011111100000011001100000000011010010000000001101001001100110011110010010110001111001010010100111100101010100011110011000011001111000101101000111100110000110110100100001111001100110000000001011010011001100110100100001111001100111100001100110011000000000000111111110000001111001001011010010110111111111001100111001100000111111111111111111000
bits stuffed: 0
['000111111111111111111000000011111010101000111100110000110011001100000000011001100101010101100110111111110110011001011010011001100110011000110011111100000011001100000000011010010000000001101001001100110011110010010110001111001010010100111100101010100011110011000011001111000101101000111100110000110110100100001111001100110000000001011010011001100110100100001111001100111100001100110011000000000000111111110000001111001001011010010110111111111001100111001100000111111111111111111000']


In [322]:
#######################
# Sequence to Transmit
#######################

sequence = ''
for fram in framed_frames:
    sequence += fram
    # print(len(fram))
    
print(len(sequence))

480


In [323]:
#######################
# Step 5: Error
#######################

import random

lim = {0.1: 9,
       0.01: 99}

rxbits = ''
un = 0

for bit in sequence:
    #rnum = random.randint(0,lim[prob])
    rnum = 1 # no errors
    if rnum == 0: # error bit
        un += 1
        if bit == '1':
            bit = '0'
        else:
            bit = '1'
    rxbits += bit
print(un)
print(len(rxbits))

0
480


In [324]:
#######################
# Step 5: Unstuffing
#######################

# stuff_seq = '000111110'

def unstuff(bits):
    i = 0
    while i < len(bits)-3:
        #print(bits[i:i+19])
        if bits[i:i+18] == '111111111111111101':
            print(bits[i:i+18])
            bits = bits[:i+16] + bits[i+17:] # remove bit stuffing
            print(bits[i:i+18])
            i += 19
        else:
            i += 8
    return bits

rxbits = unstuff(rxbits)
# rxbits, un = unstuff(sequence)
print(len(rxbits))

480


In [325]:
#######################
# Step 5: Fixing
#######################

corrected_bits = rxbits
# finding flags to reseparate frames
i = 0
start = -1
end = -1
new_frames = []
flag_flag = 0

while (i < len(corrected_bits)):
    if corrected_bits[i:i+24] == flag:
        flag_flag = 1
        if start != -1:
            end = i
            temp = corrected_bits[start:end]
            new_frames.append(temp)
            start = i
        else:
            # temp = corrected_bits[:i]
            # new_frames.append(temp)
            start = i
    if flag_flag:
        i += 24
        flag_flag = 0
    else:
        i += 1
new_frames.append(corrected_bits[start:])
# new_frames = new_frames[1:]

print(new_frames[-1])
print(len(new_frames[-2]))
print(new_frames)
# frames separated, unstuffed but need :: untripled, CRC checked, and then converted to text

000111111111111111111000
456
['000111111111111111111000000011111010101000111100110000110011001100000000011001100101010101100110111111110110011001011010011001100110011000110011111100000011001100000000011010010000000001101001001100110011110010010110001111001010010100111100101010100011110011000011001111000101101000111100110000110110100100001111001100110000000001011010011001100110100100001111001100111100001100110011000000000000111111110000001111001001011010010110111111111001100111001100', '000111111111111111111000']


In [326]:
#######################
# Step 5: Un - encode
#######################

# temp_rxseqs = []
# for frame in new_frames:
#     temp_frame = ""
#     count = 0
#     while(count+8 < len(frame)):
#         message = string_to_array(frame[count:count+8])
#         decoded_msg = decode_message(message, valid_codewords)
#         temp_frame += array_to_string(decoded_msg)
#         count += 8
#     temp_frame += frame[count:]
#     temp_rxseqs.append(temp_frame)
#     print(temp_frame)

# binaryout = temp_rxseqs
# print(binaryout)

In [332]:
#######################
# Step 5: Remove flag
#######################

rxseqs = []
for nf in new_frames:
    rxseqs.append(nf[24:])

print(rxseqs)
rxseqs = rxseqs[0:-1]
print(len(rxseqs[0]))
# need :: CRC checked, and then converted to text

['000011111010101000111100110000110011001100000000011001100101010101100110111111110110011001011010011001100110011000110011111100000011001100000000011010010000000001101001001100110011110010010110001111001010010100111100101010100011110011000011001111000101101000111100110000110110100100001111001100110000000001011010011001100110100100001111001100111100001100110011000000000000111111110000001111001001011010010110111111111001100111001100', '']
432


In [328]:
#######################
# Step 5: Remove removed bits
#######################

rxfinal = []
for f in rxseqs:
    if f !='':
        rxfinal.append(f)

rxseqs=rxfinal
print(rxseqs)
rxseqs = rxseqs[0:-1]
print(len(rxseqs[0]))
# need :: CRC checked, and then converted to text

['000011111010101000111100110000110011001100000000011001100101010101100110111111110110011001011010011001100110011000110011111100000011001100000000011010010000000001101001001100110011110010010110001111001010010100111100101010100011110011000011001111000101101000111100110000110110100100001111001100110000000001011010011001100110100100001111001100111100001100110011000000000000111111110000001111001001011010010110111111111001100111001100']


IndexError: list index out of range

In [333]:
#######################
# Step 5: Decode RM
#######################

temp_rxseqs = []
for frame in rxseqs:
    temp_frame = ""
    count = 0
    while(count+8 < len(frame)):
        message = string_to_array(frame[count:count+8])
        decoded_msg = decode_message(message, valid_codewords)
        temp_frame += array_to_string(decoded_msg)
        count += 8
    temp_frame += frame[count:]
    temp_rxseqs.append(temp_frame)
    print(temp_frame)

rxseqs = temp_rxseqs
print(rxseqs)

0100100101101110001000000011000100111000001101010011001100101100001000000111000001110010011011110110110101101001011011100110010101101110011101000010000001010011011101000010111000100000010011000110111111111000101111001100
['0100100101101110001000000011000100111000001101010011001100101100001000000111000001110010011011110110110101101001011011100110010101101110011101000010000001010011011101000010111000100000010011000110111111111000101111001100']


In [334]:
#######################
# Step 5: Check CRC
#######################

# Create CRC for g(D) = [D16 + D12 + D5 + 1]
crc_str = '10001000000100001' # divisor
int_crc_str = int(crc_str,2)
failed = 0
total = 0

for rxs in rxseqs:
    crc_rx = rxs[-16:]
    beginning = rxs[:-16] + "0000000000000000"
    beginning = int(beginning,2)
    crc = beginning % int_crc_str
    crc = format(crc,'016b')
    if len(crc) > 16:
        crc = crc[-16:]
    if crc != crc_rx:
        print("crc: "+str(crc)+" crc_rx: "+str(crc_rx))
        failed += 1
    total += 1

print('Number of Failed Frames with Error Probability ' + str(prob) + ': --- ' + str(failed) + ' --- \n')
print('Failed Frames: ' + str((failed / total) * 100) + '%' + ' of ' + str(total) + ' total frames recieved.')

crc: 1100100000000100 crc_rx: 1000101111001100
Number of Failed Frames with Error Probability 0.01: --- 1 --- 

Failed Frames: 100.0% of 1 total frames recieved.


In [335]:
#######################
# Convert Back to Text
#######################

all_rx_bits = ''
char_arr = []
paragraph = ''
i = 0

#print(rxseqs)

for rxs in rxseqs:
    print(len(rxs[:-15]))
    if len(rxs[:-15]) < 200:
        continue
    else:
        print("HERE")
        all_rx_bits += rxs[:-15]
    
while i < len(all_rx_bits)-7:
    char_arr.append(all_rx_bits[i:i+8])
    i += 8

for cb in char_arr:
    character = chr(int(cb,2))
    if (character!='~'):
        paragraph += character

print(paragraph)

205
HERE
In 1853, prominent St. Lo


In [268]:
# In 1853, prominent St. Louis merchant Wayman Crow and his pastor, William Greenleaf Eliot Jr., concerned about 
# the lack of institutions of higher learning in the growing midwest, led the founding of Washington University  
# in St. Louis. During the 1840s and 50s, waves of immigrants flooded into St. Louis, boosting the population of 
# the young city. With these newcomers came a pressing need for education - both industrial training and basic 
# general courses - conducted outside of normal working hours. So the first educational step of the young 
# Washington University was to establish an evening program on October 22, 1854. Over the succeeding decades, 
# the continuing education program underwent many changes. The university flourished at its location in downtown 
# St. Louis for its first 50 years, growing from an evening program to an institution offering a full slate of 
# scientific, liberal arts and classical course offerings. In time, schools of law and fine arts were added. In 
# 1891, the school acquired the St. Louis Medical College to form a medical department, which merged with the 
# Missouri Medical College in 1899.